In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import copy

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

%load_ext autoreload
%autoreload 2

In [2]:
mydateparser = lambda x: datetime.strptime(x, "%Y-%m-%d")
info = pd.read_csv('../data/snp_info.csv', index_col=0)

In [3]:
stocks_analysis = ['KO', 'TGT', 'PFE', 'MSFT', 'CVX', 'DVN', 'DAL', 'JPM', 'PEP', 'AAPL']

In [4]:
stocks = pd.read_csv('stocks_by_cluster.csv')
stocks

,cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6,cluster_7,cluster_8
0,PEP,TGT,ABT,MSFT,CVX,DVN,DAL,JPM
1,KO,TJX,PFE,AMAT,SLB,MRO,AAL,BAC
2,PG,WBA,MDT,AAPL,COP,KMI,UAL,MS
3,PM,WMT,ABBV,INTC,HAL,WMB,UNP,C
4,MDLZ,CVS,JNJ,NFLX,XOM,MGM,FAST,KEY
5,MO,M,BSX,NVDA,CAT,HPE,IPG,SCHW
6,NWL,KR,GILD,ATVI,RF,DISCA,GM,HBAN


In [5]:
idx_dict = {}
for i in range(len(stocks.index)):
    for j in range(len(stocks.columns)):
        stock = stocks.iloc[i,j]
        idx_dict[stock] = (i,j)

---------
Get Samples and Labels
---------

In [6]:
samples = pd.read_csv('samples_cluster.csv', index_col=0, header=[0,1,2], parse_dates=True, date_parser=mydateparser)
samples.head()

Cluster     cluster_1                                                       \
Asset         asset_1                                                        
Metrics         Close        RSI  k_percent  r_percent      MACD MACD_EMA9   
Date                                                                         
2016-01-22  95.849998  46.758102  39.393925 -60.606075 -0.449477 -0.355034   
2016-01-25  95.940002  47.496642  40.757622 -59.242378 -0.390672 -0.362422   
2016-01-26  96.320000  50.819987  47.158198 -52.841802 -0.318094 -0.353299   
2016-01-27  95.940002  47.360939  46.945914 -53.054086 -0.282311 -0.338775   
2016-01-28  97.449997  59.881188  78.066898 -21.933102 -0.156237 -0.301598   

Cluster                                                                  \
Asset                                                asset_2              
Metrics    Price Rate Of Change On Balance Volume      Close        RSI   
Date                                                                      
2016-01-22            -0.013990           3217800  42.060001  55.344782   
2016-01-25            -0.015394           7624800  42.160000  57.575863   
2016-01-26            -0.013317          11360000  42.080002  55.037601   
2016-01-27            -0.005803           5564900  42.090000  55.321647   
2016-01-28             0.017117          10734800  42.570000  66.903600   

Cluster           ...                   cluster_8                            \
Asset             ...                     asset_6                   asset_7   
Metrics           ...        Price Rate Of Change On Balance Volume   Close   
Date              ...                                                         
2016-01-22        ...                   -0.109310        -121219600    9.15   
2016-01-25        ...                   -0.124217        -136501300    8.80   
2016-01-26        ...                   -0.129617        -124863700    8.05   
2016-01-27        ...                   -0.088777        -135186600    8.26   
2016-01-28        ...                   -0.117354        -147285700    8.48   

Cluster                                                          \
Asset                                                             
Metrics           RSI  k_percent  r_percent      MACD MACD_EMA9   
Date                                                              
2016-01-22  20.659193  25.892845 -74.107155 -0.166997 -0.107536   
2016-01-25  15.455201  10.267876 -89.732124 -0.194758 -0.125617   
2016-01-26   9.523649   7.801429 -92.198571 -0.258863 -0.153038   
2016-01-27  19.504577  16.475110 -83.524890 -0.291544 -0.181377   
2016-01-28  28.975531  25.291817 -74.708183 -0.299122 -0.205358   

Cluster                                            
Asset                                              
Metrics    Price Rate Of Change On Balance Volume  
Date                                               
2016-01-22            -0.089552         -59805200  
2016-01-25            -0.128713        -131099000  
2016-01-26            -0.209234        -219455100  
2016-01-27            -0.152820        -182006200  
2016-01-28            -0.139086        -154026100  

[5 rows x 448 columns]

In [7]:
labels = pd.read_csv('labels_cluster.csv', index_col='Date', parse_dates=True, date_parser=mydateparser)
labels.head()

,PEP,KO,PG,PM,MDLZ,MO,NWL,TGT,TJX,WBA,...,FAST,IPG,GM,JPM,BAC,MS,C,KEY,SCHW,HBAN
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-22,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-01-25,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2016-01-26,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2016-01-27,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2016-01-28,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


------------------------------------
Tensor Ensemble Learning
------------------------------------

In [8]:
from GRTEL.decomposition import GLTD
from GRTEL.utils import downturn_confidence, print_scores, print_1_percentage, confusion_matrix_metrics
from GRTEL.classification import GRTEL

from hottbox.core import Tensor, TensorTKD
from hottbox.pdtools import pd_to_tensor
from hottbox.algorithms.decomposition import HOSVD, HOOI
from hottbox.utils.generation import residual_tensor
from hottbox.algorithms.classification import TelVI

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [9]:
X = []
for i in range(len(samples)):
    X_t = samples.iloc[i].reorder_levels(['Metrics', 'Asset', 'Cluster']) #Make 'Cluster' the third mode of the tensor
    X.append(pd_to_tensor(X_t))

y = np.array(labels)

print(X[0], '\n\n', y[0])

This tensor is of order 3 and consists of 448 elements.
Sizes and names of its modes are (8, 7, 8) and ['Metrics', 'Asset', 'Cluster'] respectively. 

 [1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 1. 0.
 0. 1. 0. 0. 0. 0. 0. 0.]


In [10]:
# Represent each sample in Tucker form and store it in a list
algo = HOOI()
# algo = GLTD()
rank = (4,3,3)
X_tk = [algo.decompose(sample, rank=rank) for sample in X]


# Split into train and test set
X_train, X_test, y_train, y_test = train_test_split(X_tk, y, test_size=0.25, random_state=42)

# test_size = 0.25
# k = int(len(X_tk) * (1. - test_size))

# X_train, X_test = X_tk[:k], X_tk[k:]
# y_train, y_test = y[:k], y[k:]

In [11]:
# Initialise classifier
R = np.sum(rank) # number of base classifiers required per class
n_classes = 1 if y.ndim == 1 else y.shape[1]

base_clfs = []
for _ in range(n_classes):
    base_clfs.append([DecisionTreeClassifier() for _ in range(R)])

grtel = GRTEL(base_clfs=base_clfs,
              n_classes=n_classes,
              probability=True,
              verbose=False)


# Train classifer
grtel.fit(X_train, y_train)

0 - 1 - 2 - 3 - 4 - 5 - 6 - 7 - 8 - 9 - 10 - 11 - 12 - 13 - 14 - 15 - 16 - 17 - 18 - 19 - 20 - 21 - 22 - 23 - 24 - 25 - 26 - 27 - 28 - 29 - 30 - 31 - 32 - 33 - 34 - 35 - 36 - 37 - 38 - 39 - 40 - 41 - 42 - 43 - 44 - 45 - 46 - 47 - 48 - 49 - 50 - 51 - 52 - 53 - 54 - 55 - 


In [12]:
### Classifier performance results
# train accuracy
train_score = grtel.score(X_train, y_train)
train_accuracy_df = pd.DataFrame(np.array(train_score).reshape(stocks.shape[1], stocks.shape[0])).T
train_accuracy_df.columns = stocks.columns

# test accuracy
test_score = grtel.score(X_test, y_test)
test_accuracy_df = pd.DataFrame(np.array(test_score).reshape(stocks.shape[1], stocks.shape[0])).T
test_accuracy_df.columns = stocks.columns

# confusion matrices
conf_matrices = grtel.confusion_matrices(X_test, y_test)
conf_matrices = np.array(conf_matrices).reshape(stocks.shape[1], stocks.shape[0], 2, 2)



## Print performance results
results = pd.DataFrame(columns=stocks_analysis, index=['train acc', 'test acc', 'precision', 'recall', 'downturn_precision', 'specificity'], dtype=float)
for stock in stocks_analysis:
    idx = idx_dict[stock]
    results[stock]['train acc'] = train_accuracy_df.iloc[idx]
    results[stock]['test acc'] = test_accuracy_df.iloc[idx]
    
    conf_matrix = conf_matrices[idx[1],idx[0]]
    accuracy, precision, recall, specificity, downturn_precision = confusion_matrix_metrics(conf_matrix)
    
    results[stock]['precision'] = precision
    results[stock]['recall'] = recall
    results[stock]['downturn_precision'] = downturn_precision
    results[stock]['specificity'] = specificity
    
results

,KO,TGT,PFE,MSFT,CVX,DVN,DAL,JPM,PEP,AAPL
train acc,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
test acc,0.715847,0.704918,0.748634,0.759563,0.754098,0.770492,0.732240,0.808743,0.732240,0.754098
precision,0.718182,0.693878,0.754545,0.753623,0.793814,0.875000,0.730000,0.784000,0.734513,0.761062
recall,0.790000,0.739130,0.813725,0.912281,0.754902,0.583333,0.768421,0.924528,0.813725,0.826923
downturn_precision,0.712329,0.717647,0.739726,0.777778,0.709302,0.663866,0.734940,0.862069,0.728571,0.742857
specificity,0.626506,0.670330,0.666667,0.507246,0.753086,0.908046,0.693182,0.649351,0.629630,0.658228


In [13]:
#Scores
score = grtel.score(X_train, y_train)
print("\nClassification accuracy (Train):")
print_scores(score); print()

score = grtel.score(X_test, y_test)
print("Classification accuracy (Test):")
print_scores(score); print()

print("Percentage of 1s (Test):")
print_1_percentage(y_test, n_classes); print()


Classification accuracy (Train):
[100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%]

Classification accuracy (Test):
[73.22%, 71.58%, 74.32%, 73.77%, 78.14%, 78.69%, 77.05%, 70.49%, 69.40%, 73.22%, 74.86%, 74.32%, 73.22%, 77.60%, 71.04%, 74.86%, 74.32%, 77.60%, 73.22%, 75.96%, 71.58%, 75.96%, 75.96%, 75.41%, 73.77%, 73.77%, 81.42%, 76.50%, 75.41%, 77.60%, 78.14%, 80.33%, 78.14%, 73.77%, 74.32%, 77.05%, 72.68%, 80.87%, 74.32%, 73.22%, 76.50%, 74.86%, 73.22%, 80.33%, 73.77%, 73.77%, 75.96%, 70.49%, 82.51%, 80.87%, 76.50%, 77.05%, 81.97%, 77.

-------
Grid Search
------

In [14]:
max_features = ['auto', 'sqrt', None, 'log2']
max_depth = list(range(10, 70, 10))
max_depth.append(None)
min_samples_split = [2, 5, 10, 20, 30]
min_samples_leaf = [1, 3, 5, 7, 12, 14]

search_grid = {'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

search_params = [search_grid for _ in range(R)]

print("\tPerforming grid search for each base classifer and for each class")
grtel.grid_search(X_train, y_train, search_params)

print("\tTrain base classifiers with optimal hyperparameters")
grtel.fit(X_train, y_train); print()

	Performing grid search for each base classifer and for each class
0 - 1 - 2 - 3 - 4 - 5 - 6 - 7 - 8 - 9 - 10 - 11 - 12 - 13 - 14 - 15 - 16 - 17 - 18 - 19 - 20 - 21 - 22 - 23 - 24 - 25 - 26 - 27 - 28 - 29 - 30 - 31 - 32 - 33 - 34 - 35 - 36 - 37 - 38 - 39 - 40 - 41 - 42 - 43 - 44 - 45 - 46 - 47 - 48 - 49 - 50 - 51 - 52 - 53 - 54 - 55 - 
	Train base classifiers with optimal hyperparameters
0 - 1 - 2 - 3 - 4 - 5 - 6 - 7 - 8 - 9 - 10 - 11 - 12 - 13 - 14 - 15 - 16 - 17 - 18 - 19 - 20 - 21 - 22 - 23 - 24 - 25 - 26 - 27 - 28 - 29 - 30 - 31 - 32 - 33 - 34 - 35 - 36 - 37 - 38 - 39 - 40 - 41 - 42 - 43 - 44 - 45 - 46 - 47 - 48 - 49 - 50 - 51 - 52 - 53 - 54 - 55 - 



In [15]:
### Classifier performance results
# train accuracy
train_score = grtel.score(X_train, y_train)
train_accuracy_df = pd.DataFrame(np.array(train_score).reshape(stocks.shape[1], stocks.shape[0])).T
train_accuracy_df.columns = stocks.columns

# test accuracy
test_score = grtel.score(X_test, y_test)
test_accuracy_df = pd.DataFrame(np.array(test_score).reshape(stocks.shape[1], stocks.shape[0])).T
test_accuracy_df.columns = stocks.columns

# confusion matrices
conf_matrices = grtel.confusion_matrices(X_test, y_test)
conf_matrices = np.array(conf_matrices).reshape(stocks.shape[1], stocks.shape[0], 2, 2)



## Print performance results
results = pd.DataFrame(columns=stocks_analysis, index=['train acc', 'test acc', 'precision', 'recall', 'downturn_precision', 'specificity'], dtype=float)
for stock in stocks_analysis:
    idx = idx_dict[stock]
    results[stock]['train acc'] = train_accuracy_df.iloc[idx]
    results[stock]['test acc'] = test_accuracy_df.iloc[idx]
    
    conf_matrix = conf_matrices[idx[1],idx[0]]
    accuracy, precision, recall, specificity, downturn_precision = confusion_matrix_metrics(conf_matrix)
    
    results[stock]['precision'] = precision
    results[stock]['recall'] = recall
    results[stock]['downturn_precision'] = downturn_precision
    results[stock]['specificity'] = specificity
    
results

,KO,TGT,PFE,MSFT,CVX,DVN,DAL,JPM,PEP,AAPL
train acc,0.994536,1.000000,0.979964,0.985428,0.994536,0.987250,0.998179,0.970856,1.000000,0.985428
test acc,0.672131,0.677596,0.710383,0.704918,0.743169,0.754098,0.759563,0.737705,0.765027,0.721311
precision,0.687500,0.697917,0.732673,0.683099,0.724771,0.850000,0.734694,0.725806,0.777778,0.708661
recall,0.770000,0.728261,0.725490,0.850877,0.774510,0.531250,0.757895,0.849057,0.754902,0.865385
downturn_precision,0.676056,0.712644,0.658537,0.585366,0.689189,0.634146,0.729412,0.728814,0.702381,0.750000
specificity,0.578313,0.681319,0.666667,0.347826,0.629630,0.896552,0.704545,0.558442,0.728395,0.531646


In [16]:
score = grtel.score(X_train, y_train)
print("Classification accuracy (Train):")
print_scores(score); print()

score = grtel.score(X_test, y_test)
print("Classification accuracy (Test):")
print_scores(score); print()

print("Percentage of 1s (Test):")
print_1_percentage(y_test, n_classes); print()

Classification accuracy (Train):
[100.00%, 99.45%, 97.81%, 100.00%, 100.00%, 99.82%, 99.64%, 100.00%, 99.82%, 99.27%, 99.45%, 100.00%, 99.64%, 99.82%, 100.00%, 98.00%, 99.82%, 99.82%, 100.00%, 100.00%, 100.00%, 98.54%, 98.36%, 98.54%, 100.00%, 98.72%, 94.54%, 94.72%, 99.45%, 100.00%, 100.00%, 100.00%, 99.45%, 99.82%, 100.00%, 98.72%, 100.00%, 99.82%, 100.00%, 99.45%, 99.27%, 99.64%, 99.82%, 100.00%, 100.00%, 100.00%, 99.09%, 99.27%, 99.64%, 97.09%, 100.00%, 99.64%, 100.00%, 97.27%, 100.00%, 99.09%]

Classification accuracy (Test):
[76.50%, 67.21%, 73.22%, 65.03%, 72.13%, 75.96%, 74.32%, 67.76%, 71.04%, 67.76%, 70.49%, 74.86%, 73.77%, 73.77%, 74.32%, 71.04%, 67.76%, 76.50%, 72.68%, 72.68%, 72.68%, 70.49%, 76.50%, 72.13%, 73.22%, 73.77%, 72.68%, 75.41%, 74.32%, 72.68%, 77.60%, 77.05%, 75.96%, 72.68%, 74.32%, 75.41%, 72.68%, 81.97%, 68.31%, 72.13%, 71.58%, 76.50%, 75.96%, 73.22%, 69.40%, 68.85%, 71.04%, 66.67%, 79.23%, 73.77%, 72.68%, 74.32%, 75.96%, 69.95%, 76.50%, 67.76%]

Percentage of